In [25]:
import pandas as pd
import numpy as np

In [41]:
data1 = pd.read_csv('data (1).csv')
data2 = pd.read_csv('data (2).csv')
print('data1:', data1.shape)
print('data2:', data2.shape)

data1: (10377, 20)
data2: (7452, 20)


In [42]:
data1['description'] = data1['describe']
data1.columns

Index(['price', 'type', 'time', 'link', 'address', 'describe', 'title', 'area',
       'bedroom_num', 'house_type', 'toilet_num', 'direction', 'legcal_doc',
       'block_name', 'total_floor', 'housing_feature', 'city', 'district',
       'ward', 'street', 'description'],
      dtype='object')

In [43]:
data1.drop(columns=['describe'], inplace=True)

In [44]:
data1.columns

Index(['price', 'type', 'time', 'link', 'address', 'title', 'area',
       'bedroom_num', 'house_type', 'toilet_num', 'direction', 'legcal_doc',
       'block_name', 'total_floor', 'housing_feature', 'city', 'district',
       'ward', 'street', 'description'],
      dtype='object')

In [45]:
data2 = data2[data1.columns]
data2.columns == data1.columns

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True])

In [46]:
data = pd.concat([data1, data2])

In [47]:
data.columns

Index(['price', 'type', 'time', 'link', 'address', 'title', 'area',
       'bedroom_num', 'house_type', 'toilet_num', 'direction', 'legcal_doc',
       'block_name', 'total_floor', 'housing_feature', 'city', 'district',
       'ward', 'street', 'description'],
      dtype='object')

In [48]:
data.to_csv('data-full.csv', index=None)

In [49]:
from sklearn.model_selection import train_test_split as tts
train, test = tts(data, test_size=0.2)
print('train:', train.shape)
print('test:', test.shape)

train: (14263, 20)
test: (3566, 20)


In [50]:
train.to_csv('train.csv',index=None)
test.to_csv('test.csv',index=None)